<a href="https://colab.research.google.com/github/danikhan632/COVID_Vaccine_Tracker/blob/main/vaccine_weekly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow.keras as keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Model
import random
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import pandas as pd
from imgaug import augmenters as iaa
import cv2
import ntpath
import time
import datetime

In [2]:
timeDate=datetime.date.today()- datetime.timedelta(days=1)
lastWeek=timeDate- datetime.timedelta(weeks=1)

time=timeDate.strftime("%Y-%m-%d")
lastWeek=lastWeek.strftime("%Y-%m-%d")
print(time)

2021-07-06


In [3]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

In [4]:
state_rem_list=['Virgin Islands', 'Puerto Rico','United States','Guam','Federated States of Micronesia', 'American Samoa', 'Bureau of Prisons', 'Dept of Defense', 'Indian Health Svc', 'Long Term Care', 'Marshall Islands', 'Northern Mariana Islands',  'Republic of Palau', 'Veterans Health']

In [5]:
df_vax = pd.read_csv('https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/us_state_vaccinations.csv')
state_pop=pd.read_csv('https://elasticbeanstalk-us-east-1-858154033039.s3.amazonaws.com/resources/State+Populations.csv')


state_pop= state_pop.to_dict()
pop_list=list(state_pop['2018 Population'].values())

df_vax_head_rem=['total_distributed','distributed_per_hundred','total_vaccinations_per_hundred','people_vaccinated','people_fully_vaccinated','people_fully_vaccinated_per_hundred','daily_vaccinations_raw','daily_vaccinations','daily_vaccinations_per_million','share_doses_used']

df_vax=df_vax.drop(columns=df_vax_head_rem)

for rem in state_rem_list:
    df_vax = df_vax[df_vax['location'] != rem]



prev_vax=df_vax[df_vax['date'] == lastWeek]
prev_vax.sort_values(by=['location'], ascending=True)
df_vax=df_vax[df_vax['date'] == time]
df_vax=df_vax.sort_values(by=['location'], ascending=True)

In [6]:
df_vax=df_vax.to_dict()
prev_vax=prev_vax.to_dict()

rate_list= list(df_vax['people_vaccinated_per_hundred'].values())
state_list= list(df_vax['location'].values())
vax_list= list(df_vax['total_vaccinations'].values())

prev_rate_list= list(prev_vax['people_vaccinated_per_hundred'].values())
prev_vax_list= list(prev_vax['total_vaccinations'].values())

avg_pop=328032421
avg_vax=0
num_list=[]
ddx_list=[]
ddy_list=[]
open_list=[]
etc_list=[]
for i in range(len(state_list)):
    ddx_list.append(vax_list[i] -prev_vax_list[i])
    ddy_list.append(rate_list[i]-prev_rate_list[i])
    tmp = int( ((pop_list[i] - vax_list[i])*0.8)/ddx_list[i]  )
    open_list.append( (timeDate+ datetime.timedelta(weeks =tmp)).strftime("%m-%d-%y") )
    etc_list.append( ((pop_list[i] - vax_list[i])*0.8)/ddx_list[i]  )
    avg_vax+=etc_list[i]
avg_vax=(avg_vax/51)
print(avg_vax)    

d = {'state': state_list ,'80% date':open_list,'Weeks til 80%':etc_list,'Population': pop_list,'Weekly Vax': ddx_list,'Vax%': rate_list, 'Weekly Vax rate change': ddy_list}

df = pd.DataFrame(data=d)





6.265578891393716


In [7]:
df.sort_values(by=['Population'], ascending=False)

,state,80% date,Weeks til 80%,Population,Weekly Vax,Vax%,Weekly Vax rate change
4,California,05-13-21,5.617530,39776830,2639354.0,35.74,4.42
43,Texas,05-20-21,6.348082,28704330,1875005.0,29.86,3.48
9,Florida,05-20-21,6.190691,21312211,1319523.0,32.07,3.59
32,New York State,04-29-21,3.727862,19862512,1849708.0,36.16,4.75
38,Pennsylvania,05-27-21,7.467647,12823989,640784.0,35.56,3.12
13,Illinois,05-20-21,6.001299,12768320,787485.0,35.75,3.99
35,Ohio,05-13-21,5.849894,11694664,757185.0,33.37,3.73
10,Georgia,06-03-21,8.126918,10545138,590976.0,28.10,3.76
33,North Carolina,05-27-21,7.982445,10390149,510908.0,32.02,2.71
22,Michigan,05-20-21,6.521366,9991177,598479.0,32.64,3.45


In [ ]:
x_list=[]
new_list=[]
for i in range(len(state_list)):
    x_list.append( (abs(etc_list[i]-avg_vax)*pop_list[i])/10000)
    new_list.append(i+1)

ddf = {'Number':new_list,'state': state_list , 'Critical states': x_list}

dfw = pd.DataFrame(data=ddf)


dfw.sort_values(by=['Critical states'], ascending=True)

In [ ]:

dfw.sort_values(by=['Critical states'], ascending=True)

In [ ]:
print(  len(df_vax['location'])  , len(df_vax['people_vaccinated_per_hundred'])    )

In [ ]:
yest=datetime.date.today()- datetime.timedelta(days=1)
eta= yest+ datetime.timedelta(weeks =9)

print(eta)


2021-04-25


In [ ]:
int(etc_list[5])
print(state_list[5])

Colorado
